In [1]:
import pandas as pd
import sqlite3
from pathlib import Path

Path("example_3_sql_analysis").mkdir(parents=True, exist_ok=True)

CSV_PATH = "data/omie_day_ahead_prices_es_history.csv"
DB_PATH = "example_3_sql_analysis/omie_spain_prices.db"

df = pd.read_csv(CSV_PATH)

conn = sqlite3.connect(DB_PATH)
df.to_sql("omie_prices", conn, if_exists="replace", index=False)

conn.execute("CREATE INDEX IF NOT EXISTS idx_omie_prices_date ON omie_prices(date);")
conn.execute("CREATE INDEX IF NOT EXISTS idx_omie_prices_hour ON omie_prices(hour);")
conn.commit()

print("Loaded rows:", conn.execute("SELECT COUNT(*) FROM omie_prices;").fetchone()[0])
print("DB:", DB_PATH)


Loaded rows: 1752
DB: example_3_sql_analysis/omie_spain_prices.db


In [3]:
def q(sql: str) -> pd.DataFrame:
    return pd.read_sql_query(sql, conn)

q("SELECT * FROM omie_prices LIMIT 5;")


,date,hour,price_eur_mwh
0,2024-10-01,1,104.00
1,2024-10-01,2,97.68
2,2024-10-01,3,97.68
3,2024-10-01,4,94.86
4,2024-10-01,5,84.01


In [17]:
### Top 10 most expensive days (by daily average price)

q("""
WITH daily AS (
  SELECT date, AVG(price_eur_mwh) AS daily_avg
  FROM omie_prices
  GROUP BY date
)
SELECT date, ROUND(daily_avg, 2) AS daily_avg
FROM daily
ORDER BY daily_avg DESC
LIMIT 10;
""")


,date,daily_avg
0,2024-12-12,146.67
1,2024-12-13,145.29
2,2024-11-26,143.25
3,2024-12-02,143.21
4,2024-12-11,140.61
5,2024-12-03,140.43
6,2024-11-27,136.45
7,2024-11-18,136.37
8,2024-12-05,135.62
9,2024-12-04,134.22


In [19]:
### Top 10 cheapest days (by daily average price)

q("""
WITH daily AS (
  SELECT date, AVG(price_eur_mwh) AS daily_avg
  FROM omie_prices
  GROUP BY date
)
SELECT date, ROUND(daily_avg, 2) AS daily_avg
FROM daily
ORDER BY daily_avg ASC
LIMIT 10;
""")


,date,daily_avg
0,2024-11-24,10.71
1,2024-10-06,21.96
2,2024-12-08,22.19
3,2024-10-09,24.52
4,2024-10-08,31.37
5,2024-12-07,36.67
6,2024-10-16,48.84
7,2024-10-19,49.84
8,2024-10-26,50.54
9,2024-10-17,51.25


In [9]:
### Evening price premium: hours 19–22 vs the rest of the day

q("""
WITH tagged AS (
  SELECT
    date,
    CASE WHEN hour BETWEEN 19 AND 22 THEN 'Evening (19-22)' ELSE 'Other hours' END AS bucket,
    price_eur_mwh
  FROM omie_prices
)
SELECT
  bucket,
  ROUND(AVG(price_eur_mwh), 2) AS avg_price
FROM tagged
GROUP BY bucket
ORDER BY avg_price DESC;
""")


,bucket,avg_price
0,Evening (19-22),123.22
1,Other hours,85.55


In [21]:
### Days with the largest intraday price spread (volatility proxy)

q("""
WITH daily AS (
  SELECT
    date,
    MIN(price_eur_mwh) AS min_price,
    MAX(price_eur_mwh) AS max_price,
    (MAX(price_eur_mwh) - MIN(price_eur_mwh)) AS spread
  FROM omie_prices
  GROUP BY date
)
SELECT
  date,
  ROUND(min_price, 2) AS min_price,
  ROUND(max_price, 2) AS max_price,
  ROUND(spread, 2) AS spread
FROM daily
ORDER BY spread DESC
LIMIT 10;
""")


,date,min_price,max_price,spread
0,2024-11-25,3.33,159.82,156.49
1,2024-10-10,39.83,180.00,140.17
2,2024-12-09,3.85,137.21,133.36
3,2024-10-20,0.02,125.89,125.87
4,2024-11-12,3.32,128.86,125.54
5,2024-10-13,0.11,121.28,121.17
6,2024-10-03,8.00,126.98,118.98
7,2024-10-01,56.28,173.81,117.53
8,2024-10-21,64.00,181.00,117.00
9,2024-11-20,29.12,143.18,114.06


In [23]:
### Rolling 7-day average of daily prices (trend smoothing)

q("""
WITH daily AS (
  SELECT date, AVG(price_eur_mwh) AS daily_avg
  FROM omie_prices
  GROUP BY date
),
roll AS (
  SELECT
    date,
    daily_avg,
    AVG(daily_avg) OVER (
      ORDER BY date
      ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
    ) AS rolling_7d_avg
  FROM daily
)
SELECT
  date,
  ROUND(daily_avg, 2) AS daily_avg,
  ROUND(rolling_7d_avg, 2) AS rolling_7d_avg
FROM roll
ORDER BY date;
""")


,date,daily_avg,rolling_7d_avg
0,2024-10-01,98.07,98.07
1,2024-10-02,93.59,95.83
2,2024-10-03,67.45,86.37
3,2024-10-04,98.98,89.52
4,2024-10-05,60.40,83.70
...,...,...,...
68,2024-12-09,67.19,93.57
69,2024-12-10,134.22,92.68
70,2024-12-11,140.61,93.59
71,2024-12-12,146.67,95.17


In [25]:
### Extreme price outliers: top 5% most expensive hours

q("""
WITH ranked AS (
  SELECT
    date,
    hour,
    price_eur_mwh,
    PERCENT_RANK() OVER (ORDER BY price_eur_mwh) AS pr
  FROM omie_prices
)
SELECT
  date,
  hour,
  ROUND(price_eur_mwh, 2) AS price_eur_mwh
FROM ranked
WHERE pr >= 0.95
ORDER BY price_eur_mwh DESC;
""")


,date,hour,price_eur_mwh
0,2024-11-05,19,193.00
1,2024-11-05,20,186.20
2,2024-10-21,21,181.00
3,2024-12-10,19,181.00
4,2024-10-10,21,180.00
...,...,...,...
83,2024-12-02,22,154.73
84,2024-11-18,18,154.69
85,2024-12-05,19,154.55
86,2024-11-05,21,154.45


## SQL-based market views

This section uses SQL to answer concrete questions about Spanish day-ahead electricity prices.
The focus is on quick, reproducible answers that would typically be needed in day-to-day analysis or reporting.

All results are generated directly in SQL, without Python-side aggregation.

### Expensive and cheap days
Daily rankings are used to identify:
- the most expensive days, and
- the cheapest days,
based on daily average prices.
This provides a simple way to put current prices into context by comparing them against recent historical extremes.

### Evening exposure
Prices are split between:
- evening hours (19–22), and
- the rest of the day.
This highlights where price exposure tends to concentrate and helps quantify how much evenings drive overall costs.

### Unstable days
Days are ranked by intraday price spread (max minus min hourly price).
This makes it easy to spot days with unusually large price swings, which are typically the days where assumptions are most likely to fail and closer monitoring is required.

### Short-term trend
A rolling 7-day average of daily prices is used to smooth short-term noise and highlight the underlying direction of prices.
This is a lightweight way to assess whether recent movements look temporary or persistent.
